# Сводный проект "Стримчик"

# Введение
В ходе проекта будут проведены следующие исследования:
- считывание и подготовка данных по датасету с историческими данными платформы (отредактируем названия столбцов, обработаем пропуски и дубликаты, преобразуем данные в новые типы где требуется, создадим столбец с суммарными продажами во всех регионах);
- исследовательский анализ полученных данных (сгруппируем данные по годам, выясним какие платформы оказались самыми прибыльными, рассмотрим влияние отзывов пользователей и критиков на продажи одной платформы и соотнесём результаты с другими платформами, рассмотрим распределение игр по жанрам);
- составление портрета пользователей каждого региона (определим для пользователей каждого региона самые популярные платформы, самые популярные жанры а также влияет ли рейтинг ESRB на продажи в отдельном регионе);
- проверка гипотез (о средних пользовательских рейтингах разных платформ и разных жанров);

**Цель проекта** - выявить определяющие успешность игры закономерности, что позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

In [ ]:
import pandas as pd #импортируем все необходимые библиотеки и/или методы
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats as st 

## Считывание данных

In [ ]:
data = pd.read_csv('data.csv') #считываем данные и сохраняем их в датафрейм

In [ ]:
data.head() #изучаем данные

In [ ]:
data.info() #изучаем общую информацию по датафрейму

### Промежуточный вывод
В ходе этого шага исследования мы считали исходные данные из файла и сохранили их в датафрейм. Изучив их мы можем соотнести их с заданным **описанием данных**:

- Name — название игры, формат `object`
- Platform — платформа, формат `object`
- Year_of_Release — год выпуска, формат `float64`
- Genre — жанр игры, формат `object`
- NA_sales — продажи в Северной Америке (миллионы проданных копий), формат `float64`
- EU_sales — продажи в Европе (миллионы проданных копий), формат `float64`
- JP_sales — продажи в Японии (миллионы проданных копий), формат `float64`
- Other_sales — продажи в других странах (миллионы проданных копий), формат `float64`
- Critic_Score — оценка критиков (максимум 100), формат `float64`
- User_Score — оценка пользователей (максимум 10), формат `object`
- Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию, формат `object`

## Предобработка данных

### Названия столбцов

In [ ]:
data.columns = data.columns.str.lower()#приводим все названия столбцов к нижнему регистру

In [ ]:
data.columns #проверяем корректность результата

### Проверка на пропуски и дубликаты и их обработка

In [ ]:
data.isna().sum() #проверяем данные на пропуски

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.dropna(subset=['name', 'year_of_release', 'genre']) #удаляем пропуски в столбце 'name' и 'year_of_release'

In [ ]:
data['user_score'] = data['user_score'].replace('tbd', np.nan) 
#заменяем значение 'tbd' в столбце 'user_score' на NaN

In [ ]:
data['rating'] = data['rating'].fillna('RP') #заменяем пропуски в столбце 'rating' на RP - Rating Pending

### Смена типов данных

In [ ]:
data['year_of_release'] = pd.to_datetime(data['year_of_release'], format='%Y') #меняем тип данных в столбце с годом релиза

In [ ]:
for item in ['na_sales', 'eu_sales', 'jp_sales', 'other_sales']: 
    #меняем тип данных в отдельных столбцах с float64 на floar32 для экономии вычислительной памяти
    data[item] = data[item].astype('float32')

In [ ]:
data['critic_score'] = data['critic_score'].astype('float32') #меняем тип данных в столбце с оценками пользователей

In [ ]:
data['user_score'] = data['user_score'].astype('float32') #меняем тип данных в столбце с оценками пользователей

In [ ]:
data.dtypes #проверяем результат

### Добавление столбцов

In [ ]:
data['general_sales'] = data['na_sales'] + data['eu_sales'] + data['jp_sales'] + data['other_sales']
#создаём столбец с общими продажами во всех регионах

### Финальная проверка результатов предобработки

In [ ]:
for item in data.columns:
    print(f'{item} {data[item].isna().sum()}') #проверяем наличие неполных дубликатов

In [ ]:
data.head() #проверяем результат

### Промежуточный вывод
В ходе этого шага исследования мы **предобрабатывали** данные следующим образом:
- привели названия всех столбцов к нижнему регистру для удобства работы с данными;
- проверили данные на наличие пропусков и нашли их в столбцах 'name' - **2 пропуска**, 'year_of_release' - **269 пропусков**, 'genre' - **2 пропуска**, 'critic_score' - **8578 пропусков**, 'user_score' - **6701 пропуск**, 'rating' - **6766 пропусков**, дубдикатов найдено не было;
- в столбцах **'name', 'year_of_release' и 'genre'** пропуски просто удаляем, так как их очень мало относительно общего объёма данных, и при этом данные из этих столбцов очень важны для дальнейшего исследования и пропуски в них будет сложно чем-то заменить или оставить с пропусками. Скорее всего пропуски в этих столбцах вызваны человеческим фактором;
- в столбце **'user_score'** встречалось значение 'tbd'. Это сокращение означает "подлежит уточнению", то есть конкретную оценку этой игре пользователи ещё не выставили или отзывов ещё слишком мало для подсчёта оценки. Поэтому было принято решение заменить эти значения на NaN;
- в столбцах **'user_score' и 'critic_score'** пропуски оставили без изменений, так как их слишком много чтобы удалять а логичной замены найдено не было;
- в столбце **'rating'** пропуски заменяем на значение 'RP', так как именно этот рейтинг расшифровывается как "Rating Pending" (рейтинг на рассмотрении), так как можно предположить что пропуски в данных обусловлены именно тем, что рейтинг игре ещё не назначен;
- заменили типы данных где это было необходимо (в столбце **year_of_release** - на 'datetime', так как это значение даты; в столбцах **'na_sales', 'eu_sales', 'jp_sales', 'other_sales'**, **'critic_score'** с float64 на floar32 для экономии вычислительной памяти; в столбце **'user_score'** с object на float32 так как значения в этом столбце численные);
- создали столбец с общими продажами для каждой игры во всех регионах

## Исследовательский анализ данных

### Изучение статистики ежегодного выпуска игр и выбор периода исследования

In [ ]:
data_years = data.groupby('year_of_release').agg({'name': 'count'}).sort_values(by='name') #проверяем, сколько игр выпускалось в разные годы

In [ ]:
data_years.plot(title='Количество выпущенных игр ежегодно', figsize=(10, 6))
plt.xlabel('Год выпуска')
plt.ylabel('Количество игр')
plt.show()

Года за которые вышло менее 500 игр не кажутся важными для исследования, плюс это года до 2001, то есть данные старые и уже не очень актуальные, поэтому данные по этим годам мы опустим.

In [ ]:
data = data[data['year_of_release'] >= dt.datetime(2001, 1, 1)].reset_index(drop=True) #оставляем данные только с 2001 года и обновляем индексацию

In [ ]:
data.head() #проверяем результат

### Изучение изменений продаж по платформам

In [ ]:
data.pivot_table(index='platform', values='general_sales', aggfunc='sum').sort_values(by='general_sales')
#определяем общие продажи для каждой платформы

По этим данным можно составить топ-5 платформ по продажам, так как они значительно больше для этих платформ чем для остальных. Вот этот топ - **PS2, X360, Wii, PS3, DS**.

In [ ]:
platforms_top = ['DS', 'PS3', 'Wii', 'X360', 'PS2'] #выделяем топовые платформы
data_platforms_top = data.query('platform in @platforms_top') #создаём датафрейм с данными только по ним

In [ ]:
data_ds = data_platforms_top.query('platform == "DS"').groupby('year_of_release').agg({'general_sales' : 'sum'})
data_ds.plot(figsize=(10, 5), grid=True, title='Продажи платформы DS', xlabel='Год выпуска', ylabel='Общие продажи')
plt.show()

По графику выше можно заметить что продажи платформы **DS** начались в **2004** и в стартовый год было продано **20 мил. копий**, пик продаж был в **2007-2008 годах**, в районе **300 млн. проданных копий** за эти два года, а затем начался спад  и в итоге в **2012 году** было продано около **10 млн. копий**.

In [ ]:
data_ps3 = data_platforms_top.query('platform == "PS3"').groupby('year_of_release').agg({'general_sales' : 'sum'})
data_ps3.plot(figsize=(10, 5), grid=True, title='Продажи платформы PS3', xlabel='Год выпуска', ylabel='Общие продажи')
plt.show()

По графику выше можно заметить что продажи платформы **PS3** начались в **2006** и в стартовый год было продано **20 мил. копий**, пик продаж был в **2011 году**, в районе **158 млн. проданных копий**, а затем начался спад  и в итоге в **2015 году** было продано около **18 млн. копий**.

In [ ]:
data_wii = data_platforms_top.query('platform == "Wii"').groupby('year_of_release').agg({'general_sales' : 'sum'})
data_wii.plot(figsize=(10, 5), grid=True, title='Продажи платформы Wii', xlabel='Год выпуска', ylabel='Общие продажи')
plt.show()

По графику выше можно заметить что продажи платформы **Wii** начались в **2006** и в стартовый год было продано **140 мил. копий**, пик продаж был в **2009 году**, в районе **210 млн. проданных копий**, а затем начался спад  и в итоге в **2015 году** было продано около **1 млн. копий**.

In [ ]:
data_x360 = data_platforms_top.query('platform == "X360"').groupby('year_of_release').agg({'general_sales' : 'sum'})
data_x360.plot(figsize=(10, 5), grid=True, title='Продажи платформы X360', xlabel='Год выпуска', ylabel='Общие продажи')
plt.show()

По графику выше можно заметить что продажи платформы **X360** начались в **2005** и в стартовый год было продано **10 мил. копий**, пик продаж был в **2010 году**, в районе **170 млн. проданных копий**, а затем начался спад  и в итоге в **2015 году** было продано около **10 млн. копий**.

In [ ]:
data_ps2 = data_platforms_top.query('platform == "PS2"').groupby('year_of_release').agg({'general_sales' : 'sum'})
data_ps2.plot(figsize=(10, 5), grid=True, title='Продажи платформы PS2', xlabel='Год выпуска', ylabel='Общие продажи')
plt.show()

По графику выше можно заметить что продажи платформы **PS2** начались раньше **2001** и в стартовый для нашего исследования год было продано **170 мил. копий**, пик продаж был в **2004 году**, в районе **210 млн. проданных копий**, а затем начался спад и в итоге в **2010 году** было продано менее **10 млн. копий**.

In [ ]:
top5 = [data_ds, data_ps3, data_wii, data_x360, data_ps2]
counter = 0
for item in top5:
    counter += len(item)
print(f'Средний срок жизни консоли из топ-5: {counter / len(top5)} лет')

Так как средний срок взлёта платформы - 3-4 года при среднем цикле жизни 11 лет, для рассчёта перспектив на 2017 год мы можем считать актуальными данные с 2013 года. Далее рассмотрим продажи по платформам за актуальный период.

### Изучение продаж по платформам за актуальный период

In [ ]:
data = data[data['year_of_release'] >= dt.datetime(2013, 1, 1)].reset_index(drop=True) 
#оставляем в датафрейме данные только за актуальный период

In [ ]:
data['platform'].unique() #проверяем какие остались платформы

Мы можем не рассматривать графики по платформам из топ-5, так как мы уже знаем, что к 2016 году они уже на минимальной точке, в конце спада.

In [ ]:
data_ps4 = data.query('platform == "PS4"').groupby('year_of_release').agg({'general_sales' : 'sum'})
data_ps4.plot(figsize=(10, 5), grid=True, title='Продажи платформы PS4', xlabel='Год выпуска', ylabel='Общие продажи')
plt.show()

По графику можно предположить что пик платформы **PS4** уже прошёл в **2015 году**, но и в **2016** было продано **70 млн. копий**, что хоть и не близко к пиковому значению **120 млн. копий**, но и далеко от нуля, так что можно предположить что ещё несколько лет платформа будет приносить прибыль.

In [ ]:
data_3ds = data.query('platform == "3DS"').groupby('year_of_release').agg({'general_sales' : 'sum'})
data_3ds.plot(figsize=(10, 5), grid=True, title='Продажи платформы 3DS', xlabel='Год выпуска', ylabel='Общие продажи')
plt.show()

По графику можно предположить что пик платформы **3DS** уже прошёл до рассматриваемого нами периода, и в **2016** было продано менее **10 млн. копий**. По такому резкому спаду можно предположить что на следующий год продажи платформы сойдут на нет.

In [ ]:
data_xone = data.query('platform == "XOne"').groupby('year_of_release').agg({'general_sales' : 'sum'})
data_xone.plot(figsize=(10, 5), grid=True, title='Продажи платформы XOne', xlabel='Год выпуска', ylabel='Общие продажи')
plt.show()

По графику можно предположить что пик платформы **XOne** уже прошёл в **2015 году**, а в **2016** было продано **25 млн. копий**, что хоть и не близко к пиковому значению **60 млн. копий**, но и не так близко к нулю, значит ещё пару лет платформа может приносить прибыль.

In [ ]:
data_wiiu = data.query('platform == "WiiU"').groupby('year_of_release').agg({'general_sales' : 'sum'})
data_wiiu.plot(figsize=(10, 5), grid=True, title='Продажи платформы WiiU', xlabel='Год выпуска', ylabel='Общие продажи')
plt.show()

По графику можно предположить что пик платформы **WiiU** уже прошёл в **2014 году**, а в **2016** было продано менее **5 млн. копий**, и в целом общие продажи этой платформы значительно меньше продаж других платформ, поэтому можно уверенно предположить что перспектив у этой платформы нет.

In [ ]:
data_pc = data.query('platform == "PC"').groupby('year_of_release').agg({'general_sales' : 'sum'})
data_pc.plot(figsize=(10, 5), grid=True, title='Продажи платформы PC', xlabel='Год выпуска', ylabel='Общие продажи')
plt.show()

По графику можно предположить что пик платформы **PC** уже прошёл в **2014 году**, а в **2016** было продано около **5.5 млн. копий**, и в целом общие продажи этой платформы значительно меньше продаж других платформ, поэтому можно уверенно предположить что перспектив у этой платформы нет.

In [ ]:
data_psv = data.query('platform == "PSV"').groupby('year_of_release').agg({'general_sales' : 'sum'})
data_psv.plot(figsize=(10, 5), grid=True, title='Продажи платформы PSV', xlabel='Год выпуска', ylabel='Общие продажи')
plt.show()

По графику можно предположить что пик платформы **PSV** уже прошёл в **2014 году**, а в **2016** было продано около **4.5 млн. копий**, и в целом общие продажи этой платформы значительно меньше продаж других платформ, поэтому можно уверенно предположить что перспектив у этой платформы нет.

In [ ]:
data_psp = data.query('platform == "PSP"').groupby('year_of_release').agg({'general_sales' : 'sum'})
data_psp.plot(figsize=(10, 5), grid=True, title='Продажи платформы PSP', xlabel='Год выпуска', ylabel='Общие продажи')
plt.show()

По графику можно предположить что пик платформы **PSP** уже прошёл задолго до исследуемого нами периода, а в **2016** не было продано ни одной копии, и в целом общие продажи этой платформы значительно меньше продаж других платформ, поэтому можно уверенно предположить что перспектив у этой платформы нет.

In [ ]:
data.boxplot(column='general_sales', by='platform', figsize=(20, 15))
#строим диаграмму размаха для глобальных продаж в разбивке по платформам
plt.ylim(0, 10)
plt.title('Глобальные продажи игр в разбивке по платформам')
plt.xlabel('Платформа')
plt.ylabel('Глобальные продажи')
plt.show()
data.groupby('platform').agg('general_sales').describe()

Наличие выбросов значительно дальше среднего значения скорее всего говорит нам о топовых проектах, ставших хитами и вполне возможно эксклюзивами на данных платформах. Такие случаи наблюдаются на платформе **PS4**. По графикам выше можно сделать вывод что перспективы в ближайший года есть у **PS4  и в меньшей степени у XOne**.

<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера V1<a class="tocSkip"></h2>


<b>На доработку🤔:</b>
Скорректируй вывод после изменения актуального периода.

### Изучение влияния отзывов критиков и пользователей на продажи на примере платформы PS3

Для локального исследования выберем платформу PS3 и исследуем как на игроков на этой платформе влияют отзывы критиков и пользователей.

In [ ]:
data_scores_ps3 = data[data['platform'] == 'PS3'].groupby('name').agg({'critic_score' : 'mean', 'user_score' : 'mean', 'general_sales' : 'sum'})
#группируем данные для платформы PS3 по имени проекта и выбираем нужные нам столбцы

In [ ]:
data_scores_ps3.plot(x='critic_score', y='general_sales', kind='scatter', title='Зависимость продаж от оценки критиков')
#строим диаграмму рассеяния зависимости продаж от оценки критиков
plt.xlabel('Оценка критиков')
plt.ylabel('Общие продажи')
plt.show()

In [ ]:
data_scores_ps3['general_sales'].corr(data_scores_ps3['critic_score']) #рассчитываем корреляцию между этими величинами

In [ ]:
data_scores_ps3.plot(x='user_score', y='general_sales', kind='scatter', title='Зависимость продаж от оценки пользователей')
#строим диаграмму рассеяния зависимости продаж от оценки пользователей
plt.xlabel('Оценка пользователей')
plt.ylabel('Общие продажи')
plt.show()

In [ ]:
data_scores_ps3['general_sales'].corr(data_scores_ps3['user_score']) #рассчитываем корреляцию между этими величинами

По диаграммам рассеяния и подсчётам корреляций можно сделать вывод, что зависимость продаж от оценки критиков **прямая и не слабая**, с коэффициентом корреляции приблизительно равным **0.33**, а зависимость продаж от оценки пользователей **прямая но крайне слабая**, с коэффициентом корреляции приблизительно равным **0.024**, а значит отзывы критиков влияют на решения пользователей значительно больше чем другие пользователи.

### Изучение влияния отзывов критиков и пользователей на продажи на примере платформы X360

In [ ]:
data_scores_x360 = data[data['platform'] == 'X360'].groupby('name').agg({'critic_score' : 'mean', 'user_score' : 'mean', 'general_sales' : 'sum'})
#группируем данные для платформы X360 по имени проекта и выбираем нужные нам столбцы

In [ ]:
data_scores_x360.plot(x='critic_score', y='general_sales', kind='scatter', title='Зависимость продаж от оценки критиков')
#строим диаграмму рассеяния зависимости продаж от оценки критиков
plt.xlabel('Оценка критиков')
plt.ylabel('Общие продажи')
plt.show()

In [ ]:
data_scores_x360['general_sales'].corr(data_scores_x360['critic_score']) #рассчитываем корреляцию между этими величинами

In [ ]:
data_scores_x360.plot(x='user_score', y='general_sales', kind='scatter', title='Зависимость продаж от оценки пользователей')
#строим диаграмму рассеяния зависимости продаж от оценки пользователей
plt.xlabel('Оценка пользователей')
plt.ylabel('Общие продажи')
plt.show()

In [ ]:
data_scores_x360['general_sales'].corr(data_scores_x360['user_score']) #рассчитываем корреляцию между этими величинами

По диаграммам рассеяния и подсчётам корреляций можно сделать вывод, что зависимость продаж от оценки критиков **прямая и не слабая**, с коэффициентом корреляции приблизительно равным **0.35**, а зависимость продаж от оценки пользователей **обратная но крайне слабая**, с коэффициентом корреляции приблизительно равным **-0.012**, а значит отзывы критиков влияют на решения пользователей значительно больше чем другие пользователи.

### Изучение влияния отзывов критиков и пользователей на продажи на примере платформы DS

In [ ]:
data_scores_pc = data[data['platform'] == 'PC'].groupby('name').agg({'critic_score' : 'mean', 'user_score' : 'mean', 'general_sales' : 'sum'})
#группируем данные для платформы PC по имени проекта и выбираем нужные нам столбцы

In [ ]:
data_scores_pc.plot(x='critic_score', y='general_sales', kind='scatter', title='Зависимость продаж от оценки критиков')
#строим диаграмму рассеяния зависимости продаж от оценки критиков
plt.xlabel('Оценка критиков')
plt.ylabel('Общие продажи')
plt.show()

In [ ]:
data_scores_pc['general_sales'].corr(data_scores_pc['critic_score']) #рассчитываем корреляцию между этими величинами

In [ ]:
data_scores_pc.plot(x='user_score', y='general_sales', kind='scatter', title='Зависимость продаж от оценки пользователей')
#строим диаграмму рассеяния зависимости продаж от оценки пользователей
plt.xlabel('Оценка пользователей')
plt.ylabel('Общие продажи')
plt.show()

In [ ]:
data_scores_pc['general_sales'].corr(data_scores_pc['user_score']) #рассчитываем корреляцию между этими величинами

По диаграммам рассеяния и подсчётам корреляций можно сделать вывод, что зависимость продаж от оценки критиков **прямая и не слабая**, с коэффициентом корреляции приблизительно равным **0.2**, а зависимость продаж от оценки пользователей **обратная но крайне слабая**, с коэффициентом корреляции приблизительно равным **-0.094**, а значит отзывы критиков влияют на решения пользователей значительно больше чем другие пользователи.

Рассмотрев данные средних оценок и общих продаж для трёх разных платформ от трёх разных производителей можно заметить что гипотеза о сильной зависимости продаж от оценок критиков по большей части подтверждается, а вот оценки пользователей не имеют серьёзного влияния на продажи.

### Изучение распределения игр по жанрам

In [ ]:
plt.figure(figsize=(15,5))
plt.bar(data['genre'], data['general_sales']) #строим столбчатую диаграмму для продаж по жанрам
plt.title('Продажи по жанрам')
plt.xlabel('Жанры')
plt.ylabel('Продажи')
plt.show()

In [ ]:
data_genre_pivot = data.groupby('genre').agg({'name' : 'count'}).sort_values(by='name', ascending=False)
data_genre_pivot.plot(kind='bar', title='Количество проектов по жанрам', xlabel='Жанры', ylabel='Количество игр', figsize=(15,5))
plt.show()

In [ ]:
data_genre_mean = data.groupby('genre').agg({'general_sales' : 'mean'}).sort_values(by='general_sales', ascending=False)
data_genre_mean.plot(kind='bar', title='Средние продажи по жанрам', xlabel='Жанры', ylabel='Средние продажи', figsize=(15,5))
plt.show()

In [ ]:
data_genre_median = data.groupby('genre').agg({'general_sales' : 'median'}).sort_values(by='general_sales', ascending=False)
data_genre_median.plot(kind='bar', title='Медианные продажи по жанрам', xlabel='Жанры', ylabel='Медианные продажи', figsize=(15,5))
plt.show()

Проанализировав количество общих продаж по жанрам мы увидели что большее число общих продаж приходится на жанр **Action**, а наименьшее - **Strategy, Adventure, Puzzle**. Проанализировав количество игр по жанрам мы увидели что наибольшее количество игр из наших данных приходится на жанр **Action**, наименьшее - на жанр **Puzzle**. Изучив средние и медианные продажи мы можем отметить что соотношение между жанрами почти одинаковое при рассмотрении этих значений, только медианные значения ожидаемо ниже средних, так как мы уже знаем что есть выбросы, связанные с проектами-бестселлерами и неудачными проектами. Тут мы также можем отметить что наибольшие средние и медианные продажи приходятся на жанр **Shooter**, наименьшие - на жанр **Adventure**. 

### Промежуточный вывод
В ходе этого шага исследования мы:
- рассмотрели количество игр, выпущенных за разные годы и отсекли неактуальные данные, оставив только данные с **2001 года**, так как они не такие устаревшие и более объёмные;
- рассмотрели как менялись продажи по платформам. Выбрали платформы с наибольшими суммарными продажами и построили распределения по годам. Определили средний цикл активного существования платформы в **11 лет**;
- так как средний срок взлёта платформы - 3-4 года при среднем цикле жизни 11 лет, для рассчёта перспектив на 2017 год мы определили актуальными данные с 2013 года;
- определили платформы, лидирующие по продажам - **PS2, X360, Wii, PS3, DS**;
- рассмотрели влияние отзывов критиков и пользователей на продажи для платформы **PS3** и определили, что зависимость продаж от оценки критиков **прямая и достаточно сильная**, с коэффициентом корреляции приблизительно равным **0.33**, а зависимость продаж от оценки пользователей **прямая но очень слабая**, с коэффициентом корреляции приблизительно равным **0.024**, а значит отзывы критиков влияют на решения пользователей значительно больше чем другие пользователи;
- рассмотрели влияние отзывов критиков и пользователей на продажи платформы **X360** и определили, что зависимость продаж от оценки критиков **прямая и не слабая**, с коэффициентом корреляции приблизительно равным **0.35**, а зависимость продаж от оценки пользователей **обратная но крайне слабая**, с коэффициентом корреляции приблизительно равным **-0.012**, а значит отзывы критиков влияют на решения пользователей значительно больше чем другие пользователи;
- рассмотрели влияние отзывов критиков и пользователей на продажи платформы **DS** и определили, что зависимость продаж от оценки критиков **прямая и не слабая**, с коэффициентом корреляции приблизительно равным **0.2**, а зависимость продаж от оценки пользователей **обратная но крайне слабая**, с коэффициентом корреляции приблизительно равным **-0.094**, а значит отзывы критиков влияют на решения пользователей значительно больше чем другие пользователи;
- проанализировали количество общих продаж по жанрам и увидели, что большее число общих продаж приходится на жанр **Action**, а наименьшее - **Strategy, Adventure, Puzzle**;
- проанализировали количество игр по жанрам и увидели что наибольшее количество игр из наших данных приходится на жанр **Action**, наименьшее - на жанр **Puzzle**;
- изучили средние и медианные продажи и можем отметить что соотношение между жанрами почти одинаковое при рассмотрении этих значений, только медианные значения ожидаемо ниже средних, так как мы уже знаем что есть выбросы, связанные с проектами-бестселлерами и неудачными проектами. Тут мы также можем отметить что наибольшие средние и медианные продажи приходятся на жанр **Shooter**, наименьшие - на жанр **Adventure**. 

## Портрет пользователя каждого региона

### Поиск самых популярных платформ для пользователей каждого региона

In [ ]:
data_na = data.groupby('platform').agg({'na_sales' : 'sum'}).sort_values(by='na_sales').tail()
#находим 5 самых популярных платформ для пользователей NA
data_na

In [ ]:
data_eu = data.groupby('platform').agg({'eu_sales' : 'sum'}).sort_values(by='eu_sales').tail()
#находим 5 самых популярных платформ для пользователей EU
data_eu

In [ ]:
data_jp = data.groupby('platform').agg({'jp_sales' : 'sum'}).sort_values(by='jp_sales').tail()
#находим 5 самых популярных платформ для пользователей JP
data_jp

In [ ]:
plt.figure(figsize=(12, 7))
plt.subplot(2, 2, 1)
plt.plot(data_na)
plt.title('Топ-5 платформ NA')
plt.subplot(2, 2, 2)
plt.plot(data_eu)
plt.title('Топ-5 платформ EU')
plt.subplot(2, 2, 3)
plt.plot(data_jp)
plt.title('Топ-5 платформ JP')
plt.show()

Самые популярные платформы для пользователей NA (от меньших продаж к большим) - **3DS, PS3, X360, XOne, PS4**, для пользователей EU - **3DS, X360, XOne, PS3, PS4**, для пользователей JP - **WiiU, PS4, PSV, PS3, 3DS**. У пользователей NA и EU это одни и те же платформы, только в разном порядке (хотя самая популярная и менее популярная в топах сохраняются), а у JP вместо X360 и XOne появляются PSV и WiiU. 3DS, PS3, PS4 популярны во всех регионах.

### Поиск самых популярных жанров для пользователей каждого региона

In [ ]:
genre_na = data.groupby('genre').agg({'na_sales' : 'sum'}).sort_values(by='na_sales').tail()
#находим 5 самых популярных жанров для пользователей NA
genre_na

In [ ]:
genre_eu = data.groupby('genre').agg({'eu_sales' : 'sum'}).sort_values(by='eu_sales').tail()
#находим 5 самых популярных жанров для пользователей EU
genre_eu

In [ ]:
genre_jp = data.groupby('genre').agg({'jp_sales' : 'sum'}).sort_values(by='jp_sales').tail()
#находим 5 самых популярных жанров для пользователей JP
genre_jp

In [ ]:
plt.figure(figsize=(12, 7))
plt.subplot(2, 2, 1)
plt.plot(genre_na)
plt.title('Топ-5 жанров NA')
plt.subplot(2, 2, 2)
plt.plot(genre_eu)
plt.title('Топ-5 жанров EU')
plt.subplot(2, 2, 3)
plt.plot(genre_jp)
plt.title('Топ-5 жанров JP')
plt.show()

Самые популярные жанры для пользователей NA (от меньших продаж к большим) - **Misc, Role-Playing, Sports, Shooter, Action**, для пользователей EU - **Racing, Role-Playing, Sports, Shooter, Action**, для пользователей JP - **Shooter, Fighting, Misc, Action, Role-Playing**. Для NA и EU все топовые жанры кроме наименее популярной совпадают по очерёдности, только у NA на пятом месте жанр Misc, а у EU - жанр Racing. Во всех регионах встречаются Role-Playing, Shooter и action игры на разных позициях топа, а жанр Fighting встречается только в JP.

### Поиск самых популярных  рейтингов для пользователей каждого региона

In [ ]:
rating_na = data.groupby('rating').agg({'na_sales' : 'sum'}).sort_values(by='na_sales').tail()
#находим 5 самых популярных рейтингов для пользователей NA
rating_na

In [ ]:
rating_eu = data.groupby('rating').agg({'eu_sales' : 'sum'}).sort_values(by='eu_sales').tail()
#находим 5 самых популярных рейтингов для пользователей EU
rating_eu

In [ ]:
rating_jp = data.groupby('rating').agg({'jp_sales' : 'sum'}).sort_values(by='jp_sales').tail()
#находим 5 самых популярных рейтингов для пользователей JP
rating_jp

In [ ]:
plt.figure(figsize=(12, 7))
plt.subplot(2, 2, 1)
plt.plot(rating_na)
plt.title('Топ-5 рейтингов NA')
plt.subplot(2, 2, 2)
plt.plot(rating_eu)
plt.title('Топ-5 рейтингов EU')
plt.subplot(2, 2, 3)
plt.plot(rating_jp)
plt.title('Топ-5 рейтингов JP')
plt.show()

Самые популярные рейтинги для пользователей NA - **T, E10+, E, RP, M**, для пользователей EU - **T, E10+, RP, E, M**, для пользователей JP - **E10+, M, E, T, RP**. Для всех пользователей это одни и те же рейтинги, только в NA и EU они расположены почти в одинаковом порядке (за исключением **RP** и **E**, у обоих регионов они находятся на 2 и 3 местах но в разном порядке), а в JP больше всего проектов с ещё не определенным рейтингом (возможно это связано с высоким уровнем цензуры в стране и из-за этого большинство проектов проверяется гораздо тщательнее).

### Промежуточный вывод
В ходе этого шага исследования мы составили **портрет пользователя каждого региона**:
- Для пользователей NA самыми популярными платформами оказались **3DS, PS3, X360, XOne, PS4**, жанрами -  **Misc, Role-Playing, Sports, Shooter, Action**, рейтингами - **T, E10+, E, RP, M**;
- Для пользователей EU самыми популярными платформами оказались **3DS, X360, XOne, PS3, PS4**, жанрами -  **Racing, Role-Playing, Sports, Shooter, Action**, рейтингами - **T, E10+, RP, E, M**;
- Для пользователей JP самыми популярными платформами оказались **WiiU, PS4, PSV, PS3, 3DS**, жанрами -  **Shooter, Fighting, Misc, Action, Role-Playing**, рейтингами - **E10+, M, E, T, RP**.

## Проверка гипотез

### Гипотеза №1

Определяем гипотезы:
- Нулевая гипотеза: "Средние пользовательские рейтинги платформ Xbox One и PC одинаковые"
- Альтернативная гипотеза: "Средние пользовательские рейтинги платформ Xbox One и PC отличаются"

In [ ]:
data_xone = data[data['platform'] == 'XOne'] #подготавливаем данные к анализу
data_pc = data[data['platform'] == 'PC']

In [ ]:
alpha = 0.05 #уровень статистической значимости

results_h1 = st.ttest_ind(data_xone['user_score'].dropna(), data_pc['user_score'].dropna()) 
# проводим ttest для двух выборок, в качестве альтернативной используем двустороннюю гипотезу

print(f'p-value: {results_h1.pvalue}')

if results_h1.pvalue < alpha: # проверяем p-value
    print('Отвергаем нулевую гипотезу')
else:
    print('Нет оснований отвергнуть нулевую гипотезу')

Мы **не можем отвергнуть** гипотезу о том что cредние пользовательские рейтинги платформ Xbox One и PC одинаковые, так как значение p-value значительно больше уровня статистической значимости, значит принимаем гипотезу о том что cредние пользовательские рейтинги платформ Xbox One и PC одинаковые.

### Гипотеза №2

Определяем гипотезы:
- Нулевая гипотеза: "Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») одинаковые"
- Альтернативная гипотеза: "Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные"

In [ ]:
data_action = data[data['genre'] == 'Action'] #подготавливаем данные к анализу
data_sports = data[data['genre'] == 'Sports']

In [ ]:
results_h2 = st.ttest_ind(data_action['user_score'].dropna(), data_sports['user_score'].dropna()) 
# проводим ttest для двух выборок, в качестве альтернативной используем двустороннюю гипотезу

print(f'p-value: {results_h2.pvalue}')

if results_h2.pvalue < alpha: # проверяем p-value
    print('Отвергаем нулевую гипотезу')
else:
    print('Нет оснований отвергнуть нулевую гипотезу')

Мы **отвергаем** нулевую гипотезу о том что cредние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») одинаковые,  так как значение pvalue значительно меньше уровня статистической значимости, значит можно принять гипотезу о том что средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные.

### Промежуточный вывод
В ходе этого шага мы формулировали и проверяли гипотезы:
- гипотеза о том что средние пользовательские рейтинги платформ Xbox One и PC одинаковые (составили нулевую и альтернативную гипотезу как: Нулевая гипотеза: "Средние пользовательские рейтинги платформ Xbox One и PC одинаковые", Альтернативная гипотеза: "Средние пользовательские рейтинги платформ Xbox One и PC отличаются". Они были определены именно так потому что нулевая гипотеза всегда декларирует равенство, а альтернативная - неравенство. Так как в гипотезе не было требования проверить что какие-то из пользовательских рейтингов больше/меньше, а нужно было лишь проверить на наличие отличий - мы использовали двустороннюю гипотезу. Так как мы проверяли средние значение двух генеральных совокупностей, мы использовали `ttest_ind`). Мы **не смогли отвергнуть** гипотезу о том что cредние пользовательские рейтинги платформ Xbox One и PC одинаковые, так как значение p-value значительно больше уровня статистической значимости, поэтому приняли гипотезу о том что cредние пользовательские рейтинги платформ Xbox One и PC одинаковые;
- гипотеза о том что средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») одинаковые (составили нулевую и альтернативную гипотезу как: Нулевая гипотеза: "Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») одинаковые", Альтернативная гипотеза: "Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные". Они были определены именно так потому что нулевая гипотеза всегда декларирует равенство, а альтернативная - неравенство. Так как в гипотезе не было требования проверить что какие-то из пользовательских рейтингов больше/меньше, а нужно было лишь проверить на наличие отличий - мы использовали двустороннюю гипотезу. Так как мы проверяли средние значение двух генеральных совокупностей, мы использовали `ttest_ind`). Мы **отвергли** нулевую гипотезу о том что cредние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») одинаковые,  так как значение pvalue значительно меньше уровня статистической значимости, поэтому приняли гипотезу о том что средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные.

## Общий вывод
В ходе данного исследования мы выполнили следующие шаги:
- **Изучение исходных данных**: считали исходные данные из файла и сохранили их в датафрейм. Изучив их - соотнесли их с заданным **описанием данных**: Name — название игры, формат `object`, Platform — платформа, формат `object`, Year_of_Release — год выпуска, формат `float64`, Genre — жанр игры, формат `object`, NA_sales — продажи в Северной Америке (миллионы проданных копий), формат `float64`, EU_sales — продажи в Европе (миллионы проданных копий), формат `float64`, JP_sales — продажи в Японии (миллионы проданных копий), формат `float64`, Other_sales — продажи в других странах (миллионы проданных копий), формат `float64`, Critic_Score — оценка критиков (максимум 100), формат `float64`, User_Score — оценка пользователей (максимум 10), формат `object`, Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию, формат `object`;
- **Предобработка данных**: предобработали данные следующим образом: привели названия всех столбцов к нижнему регистру для удобства работы с данными; проверили данные на наличие пропусков и нашли их в столбцах 'name' - **2 пропуска**, 'year_of_release' - **269 пропусков**, 'genre' - **2 пропуска**, 'critic_score' - **8578 пропусков**, 'user_score' - **6701 пропуск**, 'rating' - **6766 пропусков**, дубдикатов найдено не было; в столбцах **'name', 'year_of_release' и 'genre'** пропуски просто удалили, так как их очень мало относительно общего объёма данных, и при этом данные из этих столбцов очень важны для дальнейшего исследования и пропуски в них будет сложно чем-то заменить или оставить с пропусками. Скорее всего пропуски в этих столбцах вызваны человеческим фактором; в столбце **'user_score'** встречалось значение 'tbd'. Это сокращение означает "подлежит уточнению", то есть конкретную оценку этой игре пользователи ещё не выставили или отзывов ещё слишком мало для подсчёта оценки. Поэтому было принято решение заменить эти значения на NaN; в столбцах **'user_score' и 'critic_score'** пропуски оставили без изменений, так как их слишком много чтобы удалять а логичной замены найдено не было; в столбце **'rating'** пропуски заменили на значение 'RP', так как именно этот рейтинг расшифровывается как "Rating Pending" (рейтинг на рассмотрении), так как можно предположить что пропуски в данных обусловлены именно тем, что рейтинг игре ещё не назначен; заменили типы данных где это было необходимо (в столбце **year_of_release** - на 'datetime', так как это значение даты; в столбцах **'na_sales', 'eu_sales', 'jp_sales', 'other_sales'**, **'critic_score'** с float64 на floar32 для экономии вычислительной памяти; в столбце **'user_score'** с object на float32 так как значения в этом столбце численные); создали столбец с общими продажами для каждой игры во всех регионах;
- **Исследовательский анализ данных**: рассмотрели количество игр, выпущенных за разные годы и отсекли неактуальные данные, оставив только данные с **2001 года**, так как они не такие устаревшие и более объёмные; рассмотрели как менялись продажи по платформам. Выбрали платформы с наибольшими суммарными продажами и построили распределения по годам. Определили средний цикл активного существования платформы в **11 лет**; так как средний срок взлёта платформы - 3-4 года при среднем цикле жизни 11 лет, для рассчёта перспектив на 2017 год мы определили актуальными данные с 2013 года; определили платформы, лидирующие по продажам - **PS2, X360, Wii, PS3, DS**; рассмотрели влияние отзывов критиков и пользователей на продажи для платформы **PS3** и определили, что зависимость продаж от оценки критиков **прямая и достаточно сильная**, с коэффициентом корреляции приблизительно равным **0.33**, а зависимость продаж от оценки пользователей **прямая но очень слабая**, с коэффициентом корреляции приблизительно равным **0.024**, а значит отзывы критиков влияют на решения пользователей значительно больше чем другие пользователи; рассмотрели влияние отзывов критиков и пользователей на продажи платформы **X360** и определили, что зависимость продаж от оценки критиков **прямая и не слабая**, с коэффициентом корреляции приблизительно равным **0.35**, а зависимость продаж от оценки пользователей **обратная но крайне слабая**, с коэффициентом корреляции приблизительно равным **-0.012**, а значит отзывы критиков влияют на решения пользователей значительно больше чем другие пользователи; рассмотрели влияние отзывов критиков и пользователей на продажи платформы **DS** и определили, что зависимость продаж от оценки критиков **прямая и не слабая**, с коэффициентом корреляции приблизительно равным **0.2**, а зависимость продаж от оценки пользователей **обратная но крайне слабая**, с коэффициентом корреляции приблизительно равным **-0.094**, а значит отзывы критиков влияют на решения пользователей значительно больше чем другие пользователи; проанализировали количество общих продаж по жанрам и увидели, что большее число общих продаж приходится на жанр **Action**, а наименьшее - **Strategy, Adventure, Puzzle**; проанализировали количество игр по жанрам и увидели что наибольшее количество игр из наших данных приходится на жанр **Action**, наименьшее - на жанр **Puzzle**; изучили средние и медианные продажи и можем отметить что соотношение между жанрами почти одинаковое при рассмотрении этих значений, только медианные значения ожидаемо ниже средних, так как мы уже знаем что есть выбросы, связанные с проектами-бестселлерами и неудачными проектами. Тут мы также можем отметить что наибольшие средние и медианные продажи приходятся на жанр **Shooter**, наименьшие - на жанр **Adventure**;
- **Составление портрета пользователя**: составили **портрет пользователя каждого региона**: для пользователей NA самыми популярными платформами оказались **3DS, PS3, X360, XOne, PS4**, жанрами -  **Misc, Role-Playing, Sports, Shooter, Action**, рейтингами - **T, E10+, E, RP, M**; для пользователей EU самыми популярными платформами оказались **3DS, X360, XOne, PS3, PS4**, жанрами -  **Racing, Role-Playing, Sports, Shooter, Action**, рейтингами - **T, E10+, RP, E, M**; для пользователей JP самыми популярными платформами оказались **WiiU, PS4, PSV, PS3, 3DS**, жанрами -  **Shooter, Fighting, Misc, Action, Role-Playing**, рейтингами - **E10+, M, E, T, RP**;
- **Проверка гипотез**: формулировали и проверяли гипотезы: гипотеза о том что средние пользовательские рейтинги платформ Xbox One и PC одинаковые (составили нулевую и альтернативную гипотезу как: Нулевая гипотеза: "Средние пользовательские рейтинги платформ Xbox One и PC одинаковые", Альтернативная гипотеза: "Средние пользовательские рейтинги платформ Xbox One и PC отличаются". Они были определены именно так потому что нулевая гипотеза всегда декларирует равенство, а альтернативная - неравенство. Так как в гипотезе не было требования проверить что какие-то из пользовательских рейтингов больше/меньше, а нужно было лишь проверить на наличие отличий - мы использовали двустороннюю гипотезу. Так как мы проверяли средние значение двух генеральных совокупностей, мы использовали `ttest_ind`). Мы **не смогли отвергнуть** гипотезу о том что cредние пользовательские рейтинги платформ Xbox One и PC одинаковые, так как значение p-value значительно больше уровня статистической значимости, поэтому приняли гипотезу о том что cредние пользовательские рейтинги платформ Xbox One и PC одинаковые; гипотеза о том что средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») одинаковые (составили нулевую и альтернативную гипотезу как: Нулевая гипотеза: "Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») одинаковые", Альтернативная гипотеза: "Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные". Они были определены именно так потому что нулевая гипотеза всегда декларирует равенство, а альтернативная - неравенство. Так как в гипотезе не было требования проверить что какие-то из пользовательских рейтингов больше/меньше, а нужно было лишь проверить на наличие отличий - мы использовали двустороннюю гипотезу. Так как мы проверяли средние значение двух генеральных совокупностей, мы использовали `ttest_ind`). Мы **отвергли** нулевую гипотезу о том что cредние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») одинаковые,  так как значение pvalue значительно меньше уровня статистической значимости, поэтому приняли гипотезу о том что средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные.

**Возможные последующие шаги**: Было бы хорошо отдельно проанализировать консоли разных типов (стационарные и портативные), так как у них должны быть абсолютно разные продажи и т.д. а также консоли разных производителей (Sony, Microsoft, Nintendo и т.д.). Также можно проанализировать соотношение продаж по жанрам к соотношению оценок пользователей по жанрам, чтобы понять какие данные более репрезентативны. Также по рейтингу можно определить средний минимальный возраст игрока для каждого жанра.